<br>
<strong><center>Instituto Superior de Engenharia de Lisboa</center></strong>
<center>Licenciatura em Engenharia Informática e Multimédia</center>
<strong><center>Codificação de Sinais Multimédia</center></strong>

<strong><center>2º Semestre de 2018/2019</center></strong>
<br>

## <center>2º Trabalho Prático</center>

<center><strong><em>Autores:</em></strong></center>
<center><em>Luís Guimarães 42346</em></center>
<center><em>Érica Pereira 42356</em></center>


<br>
<center><strong><em>Docente:</em></strong></center>
<center><em>André Lourenço</em></center>

<br>
<br>
<div style='text-align: right; margin-right:10%;'><strong>Lisboa, 14 de abril de 2019</strong></div>

In [1]:
# imports
import cv2 
import numpy as np
import matplotlib.pyplot as plt 

from os import path 
from time import time

import os
cwd = os.getcwd()

print(cwd)
#lena.tif esta no cwd

def waitKey():
    key = cv2.waitKey(10000)
    if key == 27:
        cv2.destroyAllWindows()


C:\Users\erica\Desktop\ISEL\2a02 - CSM\4ano\Trabalho 2


## Exercício 1

### Huffman

O método de Huffman é um método de compressão sem perdas, ou seja, reduz o número de bits necessários para representar cada símbolo sem perda de informação neste processo de compressão. 
O código gerado para cada símbolo é de comprimento variável, baseado numa codificação entrópica. Os símbolos com maior número de ocorrências de um dado conjunto são representados por menos bits do que os símbolos com um menor número de ocorrências.
<br>

O algoritmo teórico do método de Huffman dá-se da seguinte forma:
<br>
1. Colocar os símbolos numa lista por ordem decrescente de probabilidade.
<br>
2. Juntar os dois símbolos com menor número de ocorrência num novo símbolo, com a respectiva probabilidade igual à soma das probabilidades dos dois.
<br>
3. Inserir o novo símbolo na lista ordenada.
<br>
4. Voltar ao ponto 2 até que a lista contenha apenas um símbolo.
<br>
5. Percorrendo as listagens das junções de símbolos, desde a lista do último símbolo criado até à lista dos símbolos originais, atribuir números binários <br> (0 ou 1) aos símbolos que vão sendo separados, distinguindo um simbolo do outro.
<br>

A abordagem tomada neste trabalho, sucedeu-se de maneira a codificar os símbolos à medida que se vão dando as junções dos símbolos.

### Função <em>listagem_ocorrencias</em>

A função que implementa o método de Huffman vai ter como parâmetros de entrada um conjunto de símbolos e o número de ocorrências de cada símbolo. Para receber os parâmetros de entrada no formato pretendido, fez-se a função <em>listagem_ocorrencias</em>, que recebe um dado conjunto de simbolos de um ficheiro e devolve um array de símbolos não repetidos e um array com os respetivos números de ocorrência. Ambos os arrays devolvidos já se encontram ordenados de forma decrescente.
 

In [2]:

def listagem_ocorrencias(mensagem):
    
    simbolos = np.unique(mensagem) 
    numSimbolos = len(simbolos)
    
    ocorrencias = {}
    
    #percorre cada elemento da mensagem e incrementa o seu número de ocorrências no dicionário
    for simbolo in mensagem:
        if simbolo in ocorrencias:
            ocorrencias[simbolo] += 1
        else:
            ocorrencias[simbolo] = 1
        
    # Organiza os símbolos segundo as suas ocorrências de modo decrescente (maior frequência primeiro)
    sortedOcorrencias = sorted(ocorrencias.items(), key = lambda simbolo: simbolo[1], reverse = True)
    
    #retorna um array de simbolos e um array das respectivas ocorrencias, os dois ordenados
    return [simbolo for (simbolo, frequencia) in sortedOcorrencias], [frequencia for (simbolo, frequencia) in sortedOcorrencias]


# ------------------------- teste -------------------------
mensagem = np.fromfile("otorrino.txt", dtype = np.uint8) #conteúdo: "otorrinolaringologista"
print("File: ", mensagem)
conjunto_simbolos, ocorrencias = listagem_ocorrencias(mensagem)
print("conjunto_simbolos: " + str(conjunto_simbolos))
print("ocorrencias: " + str(ocorrencias))
print()

File:  [111 116 111 114 114 105 110 111 108  97 114 105 110 103 111 108 111 103
 105 115 116  97]
conjunto_simbolos: [111, 114, 105, 116, 110, 108, 97, 103, 115]
ocorrencias: [5, 3, 3, 2, 2, 2, 2, 2, 1]



### Função <em>gera_huffman</em>

A função <em>gera_huffman</em> gera uma tabela com um código binário para cada símbolo de um dado conjunto, usando o método de Huffman. Tem como parâmetros de entrada um conjunto de símbolos e o número de ocorrências de cada símbolo.
<br>

Para a realização do processo do Huffman, foi necessário os seguintes componentes:

 - <strong>codigosBinary/binary</strong> - é um array de bytes vazio do tamanho do número de símbolos existente, onde vão sendo adicionados 0's e 1's nos índices/posições respectivas dos símbolos, codificando-os à medida que se vão juntando. Esta posição/índice vai ser atribuída a cada símbolo no dicionário <em>indexes</em>;
 <br>
 
 - <strong>indexes</strong> - é um dicionário auxiliar que contém como 'keys' um np.arange, que atribui um índice a cada símbolo, e contém, como 'values', as respectivas ocorrências. <br>
 Este dicionário vai servir para ordenar as posições dos símbolos, atualizados com as junções, enquanto ordena as ocorrências, que são fáceis de ordenar por serem inteiros. <br>
 As 'keys' do dicionário são índices em vez dos próprios símbolos porque vão representar a posição do respetivo símbolo no array binary (servindo diretamente como um índice);
 <br>
 
 - <strong>aux</strong> - é um array bidimensional (array de arrays) que vai conter cada índice do respetivo símbolo dentro de um array, como por exemplo: <br><code>[['0'], ['1'], ['2']]</code>. <br> 
 Este array de arrays serve, caso haja índices que ultrapassem 1 dígito, para distinguir o número de dígitos que o índice tem, como por exemplo: <br><code>[['0'], ['1'], ['2', '10']]</code> em vez de <code>['0', '1', '210']</code>. Este último array iria gerar código nos índices '2', '1' e '0' do array <em>binary</em>, em vez de gerar nos índices pretendidos - '2' e '10';
 <br>
 
 - <strong>s</strong> - é um array dos símbolos atualizados com as junções. Vai servir de auxílio na manipulação das junções de símbolos do array <em>indexes</em>;
  <br>
 
 - <strong>o</strong> - é um array das ocorrências atualizadas com as junções. Irá também servir de auxílio na manipulação das somas de ocorrências do array <em>indexes</em>;
  <br>
 
 - <strong>tabela</strong> - é o dicionário final com os símbolos iniciais como 'keys' e as suas respectivas codificações como 'values'.

<br>
Para acompanhar/visualizar o processo de Huffman, foi feita a função <em>printThroughHuffman</em>, que vai imprimindo na consola os arrays e dicionários envolvidos no processo (descomentar as duas chamadas da função no método Huffman).
<br>
<br>

In [3]:
def gera_huffman(conjunto_simbolos, ocorrencias):
    
    #array onde vão ser guardadas as codificações dos símbolos
    codigosBinary = binary = [b'' for indexSimbolo in np.arange(0, len(conjunto_simbolos))]
    
    #dicionario auxiliar para ordenar os simbolos a partir das suas ocorrências
    indexes = {str(simbolo): ocorrencia for simbolo, ocorrencia in zip(np.arange(0, len(conjunto_simbolos)), ocorrencias)}
    
    #array auxiliar em que cada elemento é um array de cada símbolo onde se vão juntar os símbolos de menor frequência
    aux = [[k] for k in list(indexes.keys())]
    
    #enquanto o número de símbolos for maior que dois, agrupam-se os dois símbolos com menor frequência
    while(len(indexes) > 1): 
        
        s = list(indexes.keys())        #simbolos actualizados com a nova ordem
        o = list(indexes.values())      #ocorrencias actualizadas com a nova ordem
     
        #printThroughHuffman(indexes, s, o, aux, binary)
        
        
        #atribui o código binário aos 2 símbolos (ou aos 2 agregados de símbolos) com menor frequência 
        #(um for para atribuir '0' ao primeiro agregado de símbolos e um outro for para atribuir '1' ao segundo)
        for indice in aux[len(aux)-2]:
            binary[int(indice)] = b'0' + binary[int(indice)]
            
        for indice in aux[len(aux)-1]:
            binary[int(indice)] = b'1' + binary[int(indice)]
            #junta os últimos símbolos/agregado de símbolos (os com menor frequência)
            aux[len(aux)-2].append(indice)           
    
        #remove o último elemento do array porque foi agrupado ao penúltimo elemento
        aux = aux[:-1]
           
        #remove os símbolos que foram agrupados, para posteriormente guardar-se o novo símbolo com a nova
        #respectiva ocorrência
        indexes.pop(s[len(s)-2], None)
        indexes.pop(s[len(s)-1], None)
        
        
        #atualiza os novos simbolos(agrupados) e as novas ocorrências (as somas de probabilidades) e
        #remove o último elemento do array (que foi agrupado ao penúltimo elemento)
        s[len(s)-2] = s[len(s)-2] + s[len(s)-1] 
        s = s[:-1]

        o[len(o)-2] = o[len(o)-2] + o[len(o)-1]
        o = o[:-1]

        #guardar-se o novo símbolo com a nova respectiva ocorrência
        indexes[s[len(s)-1]] = o[len(o)-1]
        
        indexes = sorted(indexes.items(), key = lambda simbolo: simbolo[1], reverse = True)
        #transforma de lista ordenada novamente para um dicionário
        indexes = {simbolo: ocorrencia for simbolo, ocorrencia in indexes}
        
        
        # ---> Processo de ordenação dos símbolos no array aux <---
        
        #transforma a lista de símbolos do dicionário num array
        keys = np.array(list(indexes.keys()))
        
        #devolve um array de booleanos 'a', onde fica a True na posição onde um símbolo do array 'keys'
        #for igual ao último elemento do array dos símbolos 's' (s[len(s)-1] -> a última junção de simbolos feita)
        a = keys == s[len(s)-1]
        
        #np.where(array de bools) - devolve os índices das posições que estão a True
        index = np.where(a) #ou index = np.where(a == True)
        
        #vai inserir no index (dois indices porque é um tuplo) o simbolo pretendido, e fazer um "shift"
        #nos indices seguintes do array aux, sem substituir o simbolo nesse index
        aux.insert(index[0][0], aux[len(aux)-1])
        aux = aux[:-1]
    
    #printThroughHuffman(indexes, s, o, aux, binary)
    
    tabela = {simbolo: bits for simbolo, bits in zip(conjunto_simbolos, binary)}
    
    return tabela


# ------------------------- teste -------------------------
#visualizar o processo do huffman (de preferência com um exemplo pequeno)
def printThroughHuffman(indexes, s, o, aux, binary):
    print("size: " + str(len(indexes)))
    print("simbolos actualizados: " + str(s))
    print("ocorrencias actualizadas: " + str(o))
    print("indexes: " + str(indexes))
    print("aux: " + str(aux))
    print("binary: " + str(binary))
    print("--------------------------------------------")
    print()
    
mensagem = open("otorrino.txt", 'rb')
mensagem = mensagem.read()
huffman = gera_huffman(conjunto_simbolos, ocorrencias)
print("Algoritmo de Huffman - Tabela de codificação: ")
print(huffman)
#for h,r in huffman.items():
#    print(r,h)

Algoritmo de Huffman - Tabela de codificação: 
{111: b'10', 114: b'001', 105: b'010', 116: b'0000', 110: b'0001', 108: b'110', 97: b'111', 103: b'0110', 115: b'0111'}


## Exercício 2

### Função <em>codifica</em>

A função <em>codifica</em> tem como parâmetros de entrada uma sequência de símbolos e a tabela criada pelo <em>gera_huffman</em>. Esta função vai formular uma sequência de bits correspondente à mensagem codificada, que se dá através da substituição dos símbolos recebidos pelos respectivos códigos binários da tabela de Huffman.

Para este efeito, foram percorridos todos os símbolos da mensagem, guardando a sua codificação numa variável 'bits' e indo juntando os códigos nesta, iterativamente. À medida que os códigos se vão juntando, verificava-se o tamanho da variável: caso a junção dos códigos seja igual ou superior a 8 bits, guardam-se estes primeiros 8 bits numa outra variável, 'codigo', em formato de byte, e os restantes bits ficavam guardados na variável 'bits'.

Este método está susceptível a erro, pela inevitável necessidade de preencher o último byte com 0's, quando o último código do símbolo substituído não têm bits suficientes para fazer 1 byte.

### Função <em>codifica_tabela</em>

Para, posteriormente, medir o tamanho do ficheiro comprimido, fez sentido adicionar a este o tamanho da tabela do Huffman codificada, uma vez que para descodificar um ficheiro comprimido, é necessário saber a correspondência entre os símbolos e o código de huffman gerado. 

Posto isto, foi desenvolvida uma função <em> codifica_tabela </em> que recebe a tabela criada pelo <em>gera_huffman</em> e codifica cada linha da tabela com 3 bytes: um byte para cada 'key', um byte para o número de bits do código de cada 'value' e um byte para cada 'value'.
<br>
<br>


In [4]:
def codifica(mensagem, tabela):
    
    codigo = b''        
    bits = b''
    
    # percorre todos os símbolos da mensagem
    for m in np.arange(0, len(mensagem)):
        #vai adicionando a codificação de cada símbolo
        bits += tabela[mensagem[m]]
        
        # tamanho dos bits agregados em bytes
        size = int(len(bits) / 8)
        #se o tamanho for 1 byte ou maior
        if size > 0:
            
            #adiciona os 8 primeiros bits em formato de byte -> adiciona 1 byte numa iteração
            #(ou 16 primeiros bits caso size = 2, etc) -> simplesmente adiciona 2 bytes ou mais numa iteração só
            codigo += int(bits[:size * 8], 2).to_bytes(size, byteorder = 'big')  
            
            #se int(len(bits)/8) != len(bits)/8, quer dizer que o size resultou num float, ou seja, restaram
            #bits mas não chegam para fazer um byte
            if size != len(bits) / 8:
                #adiciona os bits restantes
                bits = bits[(size) * 8:]
            else:
                #size resultou em byte(s) completo(s), por isso não restam bits
                bits = b''
                
        # caso seja o último símbolo, já tenha gerado todos os bytes possíveis e ainda restem bits mas não são 
        #suficientes para fazer um byte 
        # --> adicionam-se 0's para completar 1 byte
        if m == len(mensagem) - 1 and len(bits) != 0:
            for i in np.arange(0, 8 - len(bits)):
                bits += b'0'
                
            codigo += int(bits, 2).to_bytes(1, byteorder = 'big')
            
        #desta maneira, está isento de perdas de informação, mas fica suscetível a erro por haver informação
        #a mais
            
    return codigo

def codifica_tabela(tabela):
    
    codigo = b'' 
    
    #huffmanSize = len(tabela)
    #codigo += huffmanSize.to_bytes(1, byteorder = 'big') 
    
    # k percorre as 'keys' do dicionário huffman
    # v percorre os 'values' do dicionário huffman
    for k, v in huffman.items():
        
        #adiciona cada símbolo em formato byte
        simbolo = int(k).to_bytes(1, byteorder = 'big')
        codigo += simbolo
        
        #adiciona o tamanho da codificação do símbolo, que se encontra no 'values'
        size = len(v)
        tamanho = size.to_bytes(1, byteorder = 'big')
        codigo += tamanho
        
        #adiciona o código binário do respectivo símbolo
        code = int(v, 2).to_bytes(1, byteorder = 'big')
        codigo += code
        
    return codigo


# ------------------------- teste -------------------------
mensagemCodificada = codifica(mensagem, huffman)
print("Mensagem codificada:", mensagemCodificada) 
print("Número de bits:", len(mensagemCodificada), "bytes")
print()
tabelaCodificada = codifica_tabela(huffman)
print("Tabela codificada:", tabelaCodificada) 
print("Número de bits:", len(tabelaCodificada), "bytes")
print()
MensagemTotal = tabelaCodificada + mensagemCodificada
print("Mensagem Total:", MensagemTotal) 
print("Número de bits:", len(MensagemTotal), "bytes")

Mensagem codificada: b'\x82%\r\xb9B\xd6\x998p'
Número de bits: 9 bytes

Tabela codificada: b'o\x02\x02r\x03\x01i\x03\x02t\x04\x00n\x04\x01l\x03\x06a\x03\x07g\x04\x06s\x04\x07'
Número de bits: 27 bytes

Mensagem Total: b'o\x02\x02r\x03\x01i\x03\x02t\x04\x00n\x04\x01l\x03\x06a\x03\x07g\x04\x06s\x04\x07\x82%\r\xb9B\xd6\x998p'
Número de bits: 36 bytes


## Exercício 3

### Função <em>descodifica</em>

A função <em>descodifica</em> recebe uma sequência de bytes, que corresponde à mensagem codificada, e recebe a tabela criada pelo <em>gera_huffman</em>, que corresponde ao símbolo e ao seu respectivo código binário gerado pelo algortimo de Huffman. Esta função tem como intuito interpretar a sequência de bits codificados e transformá-los de volta nos símbolos originais, retornando uma sequência de símbolos correspondente à mensagem descodificada.

Para este efeito, a sequência foi lida byte a byte, iterando cada um dos seus bits e adicionando-os a uma variável. Caso a variável seja igual a um código da tabela Huffman, guarda-se o símbolo correspondente e recomeça-se esta variável, continuando o processo de adição de bits e verificação da tabela até chegar à sequência de símbolos original.
<br>
<br>

In [5]:
def descodifica(Bytes, tabela):  
    
    #Para acessar a tabela atraves do código, trocaram-se as keys pelos values
    tabela = {codigo: simbolos for simbolos, codigo in tabela.items()}   
    
    #array de zeros, com tamanho do 'Bytes' recebido, em formato de bits
    mensagem = np.zeros(len(Bytes) * 8)
    
    indiceMsg = 0
    code = b''
    indiceBytes = 0
    
    #percorre cada bit da mensagem codificada
    for i in np.arange(0, len(Bytes) * 8):
        
        #significa que foi percorrido um byte completo e pode transformar o próximo em int
        if i%8 == 0:
            #transforma byte em int, ex: Bytes[0:1] -> isola o primeiro byte
            aux = int.from_bytes(Bytes[indiceBytes: indiceBytes + 1], byteorder = 'big')  
            #transforma int em binário (str), f'{aux:08b}' em vez de bin() porque 08b para não perder 
            # os zeros à esquerda
            byte = f'{aux:08b}'
            indiceBytes += 1
            
        #guarda cada bit no 'code', transformando de '0'/'1' (str) em b'0'/b'1' (byte)
        # i%8 - vai sempre percorrer de 0 a 7
        if byte[i%8] == '0':
            code += b'0'
        else:
            code += b'1'
            
        #verifica se o código binário recebido existe na tabela e converte de volta para o símbolo original
        #se não existir na tabela, vão continuar a ser acrescentados bits no 'code' no if anterior até que
        # haja um código igual e faz-se um restart no 'code' = b''
        if code in tabela:
            mensagem[indiceMsg] = tabela[code]
            indiceMsg += 1
            code = b''  
    
    #elimina possíveis posições que estejam 'vazias'
    mensagem = mensagem[:indiceMsg]
            
    return mensagem

# ------------------------- teste -------------------------
mensagemDescodificada = descodifica(mensagemCodificada, huffman)
print("Mensagem descodificada: ", mensagemDescodificada) 

Mensagem descodificada:  [111. 116. 111. 114. 114. 105. 110. 111. 108.  97. 114. 105. 110. 103.
 111. 108. 111. 103. 105. 115. 116.  97. 116.]


## Exercício 4

### Função <em>escrever<em>

A função <em>escrever</em> tem como objetivo escrever uma sequência de bits recebida num ficheiro. Tem como parâmetros de entrada uma sequência de bits, que corresponde à mensagem codificada, e um nome para dar ao ficheiro a ser criado.

Para este efeito, foi utilizado o método nativo <em>open</em> do python em modo 'wb' (Write Binary ou Write Bytes), que permite escrever bytes diretamente para o ficheiro com o nome passado como parâmetro, e que cria esse ficheiro caso não exista.

Para escrever o ficheiro foi percorrida a mensagem codificada, que corresponde a uma sequência de bits byte a byte, e foi escrito o byte correspondente no ficheiro até chegar ao fim da mensagem.
<br>
<br>

In [6]:
def escrever(Bytes, filename):
     
    #escreve bytes para um ficheiro
    file = open(filename, "wb")
    
    #percorre a mensagem
    for i in np.arange(0, len(Bytes)):
        #escreve no ficheiro, byte a byte, a mensagem
        file.write(Bytes[i:i+1])
    
    file.close()
    
    
# ------------------------- teste -------------------------
escrever(mensagemCodificada, "mensagemCodificada.txt")

## Exercício 5

### Função <em>ler<em>

A função <em>ler</em> recebe o nome de um ficheiro e lê uma sequência de bits, que correspondem à mensagem codificada.

Para este efeito, foi utilizado o método nativo <em>open</em> do python em modo 'rb' (Read Binary ou Read Bytes), que permite analizar o ficheiro recebido e lê-lo de x em x bytes, com o método '.read(número de bytes)'. 

Utilizou-se esta função para armazenar todos os bytes lidos do ficheiro num objecto do tipo bytes, onde a mensagem codificada é representada por uma sequência de bytes.
<br>
<br>

In [7]:
def ler(filename):
    
    #size = path.getsize(filename)
    #mensagem = np.zeros((size, ), dtype = bytes)
    
    
    #lê os bytes do ficheiro
    f = open(filename, 'rb')
    
    #objecto do tipo bytes onde vai ser armazenada toda a mensagem lida
    mensagem = b''
    
    
    #--> enquanto existirem bytes para ser lidos do ficheiro, armazena o byte na variavél mensagem e lê o próximo byte
    
    byte = f.read(1)    
    #while byte != b"":
    while byte:
        mensagem += byte      
        byte = f.read(1)
    
    
    f.close()
    
    return mensagem
    

# ------------------------- teste -------------------------
mensagemCodificada = ler("mensagemCodificada.txt")
print("Mensagem codificada: ", mensagemCodificada) 

Mensagem codificada:  b'\x82%\r\xb9B\xd6\x998p'


## Exercício 6

Teste às funções elaboradas, usando para o efeito os ficheiros:

 - Para a imagem: Usada a imagem "lena.tiff" em tons de cinzento. 

 - Para os textos: Usados os ficheiros "ubuntu_server_guideA.pdf" e "ubuntu_server_guideB.txt". 

 - Para o áudio: Usado o ficheiro "HenryMancini-PinkPanther30s.mp3". 

 - Para o midi: Usado o ficheiro "HenryMancini-PinkPantherC.mid". 

 - Para o ECG: Eletrocardiograma - usado o ficheiro "ecg.txt".


In [9]:
tipo = ['imagem', 'pdf', 'txt', 'audio', 'midi', 'ecg']

filenameOriginal = ["lena.tiff", 
                    "ubuntu_server_guideA.pdf", 
                    "ubuntu_server_guideB.txt", 
                    "HenryMancini-PinkPanther30s.mp3", 
                    "HenryMancini-PinkPantherC.mid", 
                    "ecg.txt"]


In [53]:
def teste(fileIndice):
    
    print("--> File Type:", tipo[fileIndice])
    print()
    filenameCompressed = "compressed_" + tipo[fileIndice] + ".txt"
    filenameUncompressed  = "uncompressed_" + tipo[fileIndice] + ".txt"

    file = open(filenameOriginal[fileIndice], 'rb')
    byte = file.read(1)
    mensagem = b''
    while byte:
        mensagem += byte
        byte = file.read(1)
        
    # ------------------ alínea A ------------------
    # Gere o código usando a função realizada no ponto 1. Meça o tempo que demora a função. 

    # Gera o código de Huffman 
    t0 = time() 
    conjunto_simbolos, ocorrencias = listagem_ocorrencias(mensagem)
    tabela_codigo = gera_huffman(conjunto_simbolos, ocorrencias) 
    t1 = time() 
    print("huffman time:", t1-t0)
    print()

    # ------------------ alínea B ------------------
    # Meça a entropia e o número médio de bits por símbolo. Calcule a eficiência. 

    Hs = 0
    media = 0
    size = len(tabela_codigo.values())
    for i in np.arange(0, size):
        prob = ocorrencias[i]/len(mensagem)
        Hs -= prob * np.log2(prob)
        media += len(list(tabela_codigo.values())[i]) * prob
    eficiencia = Hs / media
    print("Entropia:", Hs)
    print("Número médio de bits por símbolo:", media)
    print("Eficiência:", eficiencia)
    print()

    # ------------------ alínea C ------------------
    # Faça a codificação da mensagem contida no ficheiro (usando a função realizada no ponto 2).
    # Meça o tempo que a função demora a fazer a codificação.
    t2 = time()
    # Codifica
    seq_bit0 = codifica(mensagem, tabela_codigo) 
    tabela_codificada = codifica_tabela(tabela_codigo)
    t3 = time() 
    print("codifica time:", t3-t2)
    print()
    
    # ------------------ alínea D ------------------
    # Grave um ficheiro com a mensagem codificada, usando a função realizada no ponto 4. Veja o tamanho do ficheiro. 
    t6 = time() 
    escrever(seq_bit0, filenameCompressed) 
    t7 = time() 
    print("escrever time:", t7-t6)
    print()
    
    # ------------------ alínea E ------------------
    # Leia do ficheiro o conjunto de bits, usando a função realizada no ponto 5. 
    # Lê ficheiro e descodifica 
    t8 = time() 
    seq_bit1 = ler(filenameCompressed) 
    t9 = time() 
    print("ler time:", t9-t8)
    print()

    # ------------------ alínea F ------------------
    # Faça a descodificação da mensagem (usando a função realizada no ponto 3).
    # Meça o tempo que a função demora a fazer a descodificação.
    t4 = time() 
    yi = descodifica(seq_bit1, tabela_codigo) 
    t5 = time() 
    print("descodifica time:", t5-t4)
    print()
    
    # ------------------ alínea G ------------------
    # Compare a mensagem descodificada com a original e verifique que são iguais (erro nulo).

    size_ini = path.getsize(filenameOriginal[fileIndice]) 
    #size_end = path.getsize(filenameCompressed[fileIndice]) 
    #size_ini = len(file)
    size_end = len(yi)
    print("size_ini (Original):     ", size_ini)
    print("size_end (Descomprimida):", size_end)
    print("taxa de compressão:", 1.* size_ini / size_end)
    print()
    
    size_comp = len(seq_bit0) + len(tabela_codificada) 
    print("size_comp (Comprimida):", len(seq_bit0))
    print("size_comp com tabela:  ", size_comp)
    print("taxa de compressão:", 1.* size_ini / size_comp)
    print("-----------------------------------------------------")
    print()

for fileIndice in np.arange(0, len(tipo)):    
    teste(fileIndice)

#teste(0)

--> File Type: imagem

huffman time: 0.09372806549072266

Entropia: 7.452159120889992
Número médio de bits por símbolo: 7.474852758539244
Eficiência: 0.9969640020502977

codifica time: 1.9800314903259277

escrever time: 0.1561741828918457

ler time: 1.2015650272369385

descodifica time: 2.3669278621673584

size_ini (Original):      262494
size_end (Descomprimida): 262494
taxa de compressão: 1.0

size_comp (Comprimida): 245263
size_comp com tabela:   245290
taxa de compressão: 1.0701373883974072
-----------------------------------------------------

--> File Type: pdf



KeyboardInterrupt: 

#### Output dos testes:

--> File Type: imagem

huffman time: 0.062485694885253906

Entropia: 7.452159120889992<br>
Número médio de bits por símbolo: 7.474852758539244<br>
Eficiência: 0.9969640020502977

codifica time: 1.7835979461669922

escrever time: 0.17180252075195312

ler time: 1.3186678886413574

descodifica time: 3.5837788581848145

size_ini (Original):      262494<br>
size_end (Descomprimida): 262494<br>
taxa de compressão: 1.0

size_comp (Comprimida): 245263<br>
size_comp com tabela:   245290<br>
taxa de compressão: 1.0701373883974072
_______________________________________________________


--> File Type: pdf

huffman time: 0.26883959770202637

Entropia: 7.046711066952311<br>
Número médio de bits por símbolo: 7.066797580922372<br>
Eficiência: 0.9971576214345963

codifica time: 436.65767455101013

escrever time: 0.8041853904724121

ler time: 459.5441598892212

descodifica time: 11.883227586746216

size_ini (Original):      1469155<br>
size_end (Descomprimida): 1469155<br>
taxa de compressão: 1.0

size_comp (Comprimida): 1297778<br>
size_comp com tabela:   1297805<br>
taxa de compressão: 1.1320306209330369
_______________________________________________________


--> File Type: txt

huffman time: 0.1376333236694336

Entropia: 4.696550504756963<br>
Número médio de bits por símbolo: 4.731033434809716<br>
Eficiência: 0.9927113324122723

codifica time: 5.511301755905151

escrever time: 0.3117668628692627

ler time: 4.308479070663452

descodifica time: 4.324437618255615

size_ini (Original):      763396<br>
size_end (Descomprimida): 763396<br>
taxa de compressão: 1.0

size_comp (Comprimida): 451457<br>
size_comp com tabela:   451484<br>
taxa de compressão: 1.6908594767477918
_______________________________________________________


--> File Type: audio

huffman time: 0.15561318397521973

Entropia: 7.985323014536518<br>
Número médio de bits por símbolo: 7.995643123707053<br>
Eficiência: 0.9987092834171231

codifica time: 116.15043473243713

escrever time: 0.5206408500671387

ler time: 117.33427906036377

descodifica time: 7.211677551269531

size_ini (Original):      779228<br>
size_end (Descomprimida): 779228<br>
taxa de compressão: 1.0

size_comp (Comprimida): 778804<br>
size_comp com tabela:   778831<br>
taxa de compressão: 1.000509738312933
_______________________________________________________


--> File Type: midi

huffman time: 0.020946979522705078

Entropia: 5.98795309626177<br>
Número médio de bits por símbolo: 6.016982663530978<br>
Eficiência: 0.995175394563598

codifica time: 0.11664628982543945

escrever time: 0.03091883659362793

ler time: 0.045876264572143555

descodifica time: 0.33015990257263184

size_ini (Original):      48049<br>
size_end (Descomprimida): 48049<br>
taxa de compressão: 1.0

size_comp (Comprimida): 36139<br>
size_comp com tabela:   36166<br>
taxa de compressão: 1.3285682685395122
_______________________________________________________

--> File Type: ecg

huffman time: 0.04587864875793457

Entropia: 3.3951737461621434<br>
Número médio de bits por símbolo: 3.4469800470842498<br>
Eficiência: 0.9849705248610509

codifica time: 0.6069145202636719

escrever time: 0.06981563568115234

ler time: 0.22942376136779785

descodifica time: 1.0272552967071533

size_ini (Original):      259110<br>
size_end (Descomprimida): 259111<br>
taxa de compressão: 0.9999961406501461

size_comp (Comprimida): 111644<br>
size_comp com tabela:   111671<br>
taxa de compressão: 2.3202980182858575
_______________________________________________________

### Resultados do testes feitos aos ficheiros:

| Variáveis 	| Imagem 	| Texto (pdf) 	| Texto (txt) 	| Áudio 	| Midi 	| ECG 	|
|:----------------------------------:	|:------:	|:-----------:	|:-----------:	|:-------:	|:-----:	|:------:	|
| Tempo de Huffman 	| 0.062 	| 0.269 	| 0.138 	| 0.156 	| 0.021 	| 0.046 	|
| Tempo de Codificação 	| 1.784 	| 436.658 	| 5.511 	| 116.150 	| 0.117 	| 0.607 	|
| Tempo de Escrever Ficheiro 	| 0.172 	| 0.804 	| 0.312 	| 0.521 	| 0.031 	| 0.070 	|
| Tempo de Ler Ficheiro 	| 1.319 	| 459.544 	| 4.308 	| 117.334 	| 0.046 	| 0.229 	|
| Tempo de Descodificação 	| 3.584 	| 11.883 	| 4.324 	| 7.212 	| 0.330 	| 1.027 	|
| Entropia 	| 7.452 	| 7.047 	| 4.697 	| 7.985 	| 5.988 	| 3.395 	|
| Número Médio (bits/Símbolo) 	| 7.475 	| 7.067 	| 4.731 	| 7.996 	| 6.017 	| 3.447 	|
| Eficiência 	| 0.997 	| 0.997 	| 0.993 	| 0.999 	| 0.995 	| 0.985 	|
| Tamanho Original (To) 	| 262494 	| 1469155 	| 763396 	| 779228 	| 48049 	| 259110 	|
| Tamanho Descomprimido (Td) 	| 262494 	| 1469155 	| 763396 	| 779228 	| 48049 	| 259111 	|
| Taxa de Compressão (To/Td) 	| 1.0 	| 1.0 	| 1.0 	| 1.0 	| 1.0 	| 0.999 	|
| Tamanho Comprimido 	| 245263 	| 1297778 	| 451457 	| 778804 	| 36139 	| 111644 	|
| Tamanho Comprimido com Tabela (Tc) 	| 245290 	| 1297805 	| 451484 	| 778831 	| 36166 	| 111671 	|
| Taxa de Compressão (To/Tc) 	| 1.070 	| 1.132 	| 1.691 	| 1.001 	| 1.329 	| 2.320 	|

<center><strong>Legenda: </strong> O tempo está representado em segundos.</center>

<br>
<br>

## Conclusões 

Constatou-se que, em ficheiros de maior tamanho, como o texto em 'pdf' ou o áudio, o tempo que demora a codificar e o tempo que demora a ler o ficheiro, seja a ler o ficheiro original ou a mensagem comprimida, demoram a cima de 100 segundos para serem executados. 

A possibilidade referida anteriormente de, na codificação do ficheiro, ocorrer a adição de 0's na mensagem codificada para o preenchimento do byte final, sucedeu-se no último ficheiro testado, ECG.txt, onde a taxa de compressão entre o tamanho do ficheiro original e o tamanho final é igual a 0.999 (erro por acréscimo de um byte).


<br>
<br>